# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [6]:
import pandas as pd

reviews_sample = pd.read_csv('data/reviews_sample.csv', index_col=0)
recipes_sample = pd.read_csv('data/recipes_sample.csv')

In [53]:
recipes_sample = recipes_sample[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]
recipes_sample.head()

,id,name,minutes,submitted,description,n_ingredients
0,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,38798,i can t believe it s spinach,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,35173,italian gut busters,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,84797,love is in the air beef fondue sauces,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN


In [54]:
reviews_sample.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [56]:
reviews_sample_5_perc = reviews_sample.sample(frac=0.05)
recipes_sample_5_perc = recipes_sample.sample(frac=0.05)

In [58]:
writer = pd.ExcelWriter('recipes.xlsx', engine='xlsxwriter')

recipes_sample_5_perc.to_excel(writer, sheet_name='Рецепты')
reviews_sample_5_perc.to_excel(writer, sheet_name='Отзывы')
writer.close()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [125]:
import xlwings as xw
wb = xw.Book('recipes.xlsx')

In [126]:
recipe_sheet = wb.sheets['Рецепты']

In [127]:
minutes = recipe_sheet.range('D:D').options(pd.DataFrame, index=False).value
minutes

,minutes
0,15.0
1,25.0
2,25.0
3,28.0
4,45.0
...,...
1048570,NaN
1048571,NaN
1048572,NaN
1048573,NaN


In [128]:
minutes.columns = ['seconds_assign']
seconds = minutes * 60
seconds

,seconds_assign
0,900.0
1,1500.0
2,1500.0
3,1680.0
4,2700.0
...,...
1048570,NaN
1048571,NaN
1048572,NaN
1048573,NaN


In [129]:
recipe_sheet.range('H:H').options(index=False).value = seconds

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [130]:
recipe_sheet.range('I:I').formula = '=D1*60'
recipe_sheet.range('I1').value = 'seconds_formula'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [131]:
from xlwings.constants import VAlign, HAlign
recipe_sheet.range('A1:J1').api.Font.Bold = True
recipe_sheet.range('A1:J1').api.HorizontalAlignment = HAlign.xlHAlignCenter

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [132]:
# xw.Range('A1').color = (255, 0, 0)

for index, minutes in enumerate(recipe_sheet.range('D2').expand('down').value):
    place = 'D' + str(index + 2)
    if minutes > 10:
        recipe_sheet.range(place).color = (255, 0, 0)
    elif minutes < 5:
        recipe_sheet.range(place).color = (0, 255, 0)
    else:
        recipe_sheet.range(place).color = (255, 255, 0)

In [119]:
recipe_sheet.range('D2').expand('down').value

[15.0,
 25.0,
 25.0,
 28.0,
 45.0,
 60.0,
 120.0,
 25.0,
 3.0,
 15.0,
 75.0,
 5.0,
 35.0,
 30.0,
 20.0,
 70.0,
 25.0,
 5.0,
 35.0,
 35.0,
 45.0,
 60.0,
 40.0,
 10.0,
 45.0,
 25.0,
 10.0,
 10.0,
 40.0,
 40.0,
 50.0,
 60.0,
 30.0,
 5.0,
 15.0,
 370.0,
 70.0,
 35.0,
 80.0,
 45.0,
 35.0,
 10.0,
 50.0,
 28.0,
 21.0,
 30.0,
 75.0,
 30.0,
 60.0,
 5.0,
 50.0,
 120.0,
 18.0,
 75.0,
 320.0,
 48.0,
 40.0,
 167.0,
 20.0,
 25.0,
 10.0,
 15.0,
 2.0,
 13.0,
 5.0,
 15.0,
 15.0,
 270.0,
 25.0,
 105.0,
 52.0,
 20.0,
 30.0,
 25.0,
 135.0,
 25.0,
 55.0,
 25.0,
 50.0,
 85.0,
 11.0,
 40.0,
 25.0,
 40.0,
 45.0,
 16.0,
 250.0,
 40.0,
 5.0,
 105.0,
 110.0,
 1.0,
 37.0,
 510.0,
 40.0,
 80.0,
 20.0,
 15.0,
 22.0,
 40.0,
 20.0,
 20.0,
 20.0,
 1.0,
 5.0,
 70.0,
 35.0,
 5.0,
 40.0,
 40.0,
 50.0,
 65.0,
 165.0,
 6.0,
 120.0,
 12.0,
 225.0,
 80.0,
 30.0,
 65.0,
 30.0,
 60.0,
 20.0,
 125.0,
 30.0,
 20.0,
 57.0,
 55.0,
 45.0,
 32.0,
 40.0,
 125.0,
 370.0,
 32.0,
 35.0,
 35.0,
 10.0,
 15.0,
 75.0,
 315.0,
 60.0,
 30.0,


7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [180]:
# count = len(recipe_sheet.range('A2').expand('down'))

In [181]:
# recipe_sheet.range('J2').value = "=COUNTIF(Worksheets(1)!С:С, B2)"

In [182]:
# for i in range(count):
#     recipe_sheet.range(f'J{i + 2}').value = "=COUNTIF('Отзывы'!C:C; 'Рецепты'!B2)"

In [183]:
# recipe_sheet.range(f'J2:J{count}').formula = "=СЧЁТЕСЛИ(Отзывы!C:C; Рецепты!B2)"

In [184]:
wb.save()
wb.close()

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [185]:
import openpyxl
import pandas as pd

def validate(file_path):
    # загрузка данных из excel файла
    wb = openpyxl.load_workbook(filename=file_path)
    reviews = pd.read_excel(file_path, sheet_name='Отзывы')
    recipes = pd.read_excel(file_path, sheet_name='Рецепты')
    invalid_reviews = []
    
    # проверка каждого отзыва на соответствие правилам
    for i, row in reviews.iterrows():
        rating = row['Рейтинг']
        recipe_id = row['Рецепт']
        
        # проверка соответствия рейтинга диапазону от 0 до 5
        if rating < 0 or rating > 5:
            invalid_reviews.append(i+2) # добавляем строку в список с недопустимым рейтингом
        
        # проверка наличия рецепта в листе "Рецепты"
        if recipe_id not in recipes['ID'].values:
            invalid_reviews.append(i+2) # добавляем строку в список с несуществующим рецептом
    
    # выделение строк с ошибками
    if invalid_reviews:
        for row in invalid_reviews:
            reviews.style.applymap(lambda x: 'background-color: yellow', subset=[reviews.columns[0]]).iloc[row-2:row-1].applymap(lambda x: 'background-color: red')
        
    return reviews

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 